<a href="https://colab.research.google.com/github/ritaalamino/candc_master/blob/main/CANDC_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## *CANDC*

Funções não suportadas

*   --semantics
*   --modal
*   --elimeq não muda nada
*   --vpe  não muda nada
*   --robust  não muda nada
*   --roles  não muda nada

##Importa bibliotecas

In [ ]:
# Import das bibliotecas.

# Biblioteca de logging
import logging
import subprocess
import os
import csv

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Instala as libraries necessárias

In [ ]:
# Diretório do cohebert
DIRETORIO_CANDC = "CANDC"

# Diretório local para os arquivos pré-processados
DIRETORIO_LOCAL = "/content/" + DIRETORIO_CANDC + "/"

# Diretório no google drive com os arquivos pré-processados
DIRETORIO_DRIVE = "/content/drive/MyDrive/Colab Notebooks/Data/" + DIRETORIO_CANDC + "/"

In [ ]:
!git clone https://github.com/valeriobasile/learningbyreading.git

Cloning into 'learningbyreading'...
remote: Enumerating objects: 1870, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 1870 (delta 30), reused 58 (delta 22), pack-reused 1797
Receiving objects: 100% (1870/1870), 295.87 MiB | 14.35 MiB/s, done.
Resolving deltas: 100% (722/722), done.
Updating files: 100% (970/970), done.


In [ ]:
os.chdir('./learningbyreading/ext')

In [ ]:
!./install_candc.sh

A saída de streaming foi truncada nas últimas 5000 linhas.
g++ -W -Wall -Wextra -fPIC -O3 -Isrc/include -isystem ext/include   -c -o src/lib/io/writer_multi_vert.o src/lib/io/writer_multi_vert.cc
g++ -W -Wall -Wextra -fPIC -O3 -Isrc/include -isystem ext/include   -c -o src/lib/io/writer_format.o src/lib/io/writer_format.cc
g++ -W -Wall -Wextra -fPIC -O3 -Isrc/include -isystem ext/include   -c -o src/lib/io/writer_factory.o src/lib/io/writer_factory.cc
g++ -W -Wall -Wextra -fPIC -O3 -Isrc/include -isystem ext/include   -c -o src/lib/tagger/tagdict.o src/lib/tagger/tagdict.cc
g++ -W -Wall -Wextra -fPIC -O3 -Isrc/include -isystem ext/include   -c -o src/lib/tagger/tagsetdict.o src/lib/tagger/tagsetdict.cc
g++ -W -Wall -Wextra -fPIC -O3 -Isrc/include -isystem ext/include   -c -o src/lib/tagger/_baseimpl.o src/lib/tagger/_baseimpl.cc
In file included from src/include/config/options.h:15,
                 from src/include/config/config.h:12,
                 from src/include/tagger/_baseimpl

In [ ]:
# # Gera boxer em xml
# command = "./install_candc.sh"
# subprocess.run(command, shell=True, check=True, capture_output=True)

In [ ]:
# Import das bibliotecas.
import sys # Biblioteca para acessar módulos do sistema

# Se estiver executando no Google Colaboratory
# Retorna true ou false se estiver no Google Colaboratory
IN_COLAB = "google.colab" in sys.modules

In [ ]:
# Importando as bibliotecas.
import os

# Cria o diretório para receber os arquivos Originais e Permutados
# Diretório a ser criado
dirbase = DIRETORIO_LOCAL[:-1]

if not os.path.exists(dirbase):
    # Cria o diretório
    os.makedirs(dirbase)
    logging.info("Diretório criado: {}".format(dirbase))
else:
    logging.info("Diretório já existe: {}".format(dirbase))

## Verifica se arquivos estão corretos

In [ ]:

os.chdir('/content/learningbyreading/ext/candc')
!ls


bin	    lib		       Makefile.cygwin	 Makefile.mingw    models	     RELEASE.txt
doc	    LICENCE-BOXER.txt  Makefile.deps	 Makefile.sunos    models-1.02.tbz2  share
ext	    LICENCE.txt        Makefile.macosx	 Makefile.targets  pl-6.6.6	     src
grepsource  Makefile	       Makefile.macosxu  Makefile.unix	   pl-6.6.6.tar.gz   working


In [ ]:
#!chmod +x bin/candc
!bin/candc --version
!bin/boxer --version

candc v1.00 (unix build on 12 February 2024, 22:42:33)
boxer v1.00 (unix build on 12 February 2024, 22:46:41)


Roda teste

#Importa word2vec

In [ ]:

import re
import numpy as np

In [ ]:
from nltk.corpus import wordnet as wn
from scipy.spatial.distance import cosine
from sklearn.feature_extraction.text import TfidfVectorizer

#Carrega word2vec

In [ ]:
# from gensim.models import KeyedVectors
# import gensim.downloader as api

# # Carregar o modelo Word2Vec
# word2vec_model = api.load('glove-wiki-gigaword-300')

In [ ]:
# word2vec_model.init_sims(replace=True)

#Carrega bert

In [ ]:
from transformers import BertTokenizer, BertModel
import torch

# Inicializa o tokenizer e o modelo
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased',
                                          do_lower_case=False)
model = BertModel.from_pretrained('bert-base-multilingual-cased',
                                  output_attentions=False,
                                  output_hidden_states=True)

# Função para obter embeddings do BERT
def get_bert_embeddings(text):
    # Tokeniza a entrada e converte para tensor
    inputs = tokenizer(text, return_tensors="pt")

    # Obtém os embeddings
    with torch.no_grad():
        outputs = model(**inputs)

    # Retorna os embeddings da última camada
    return outputs.last_hidden_state

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

In [ ]:
# from transformers import BertTokenizer, BertModel
# import torch

# # Initialize the tokenizer and the model
# tokenizer = BertTokenizer.from_pretrained('bert-large-cased')
# model = BertModel.from_pretrained('bert-large-cased')

# # Set the model to evaluation mode
# model.eval()

# # Function to obtain BERT embeddings
# def get_bert_embeddings(text_list):
#     # Ensure text_list is not empty
#     if not text_list:
#         return None

#     # Tokenize the input batch and convert to tensors
#     inputs = tokenizer(text_list, return_tensors="pt", padding=True, truncation=True)

#     # Obtain the embeddings
#     with torch.no_grad():
#         outputs = model(**inputs)

#     # Return the embeddings from the last layer
#     return outputs.last_hidden_state

# # Example usage
# text_list = ["This is an example sentence.", "Another sentence for illustration."]
# embeddings = get_bert_embeddings(text_list)
# print(embeddings.shape)  # Print the shape of the embeddings tensor

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

torch.Size([2, 8, 1024])


In [ ]:

# Exemplo de uso
embeddings = get_bert_embeddings("Hello, world!")
print(embeddings)

tensor([[[-0.4218, -0.2564,  0.1639,  ..., -0.2528,  0.2395,  0.0357],
         [-0.6650, -0.1311,  0.6369,  ..., -0.5864,  0.1911, -0.4358],
         [-0.5932, -0.1694,  0.0448,  ..., -0.6263,  0.3855, -0.0178],
         [-0.9403, -0.0142,  0.4910,  ..., -1.0551,  0.5379, -0.7017],
         [-0.5232, -0.2376,  0.0461,  ..., -0.4888,  0.2440, -0.0680],
         [-0.4428, -0.2193,  0.4927,  ..., -0.6848,  0.2164, -0.0179]]])


# Importa Paragram300




In [ ]:
# from gensim.models import KeyedVectors

# # Path to the downloaded model files
# model_path = DIRETORIO_DRIVE + 'paragram-300-sl999.model'

# # # Carregar o modelo Word2Vec
# word2vec_model = KeyedVectors.load(model_path)


In [ ]:
# word2vec_model.init_sims(replace=True)

#Define função do grafo

In [ ]:

class Graph:
    def __init__(self, sentence: str, debug: bool = False):
        self.words = sentence.split(" ")
        self.count = len([word for word in self.words if len(word) > 0])
        self.gLen = self.count
        self.nodes = [None] * (self.count * 2)  # Armazenamento para conceitos
        self.edges = [None] * (self.count * 2)  # Armazenamento para arestas
        self.conceptCount = 0  # Contador para conceitos
        self.edgeCount = 0  # Contador para arestas
        self.repeated = 0
        self.autoInc = 0  # Variável para auto-incremento
        self.debug = debug  # Variável para debug

        if debug:
            print(sentence)

    def get_vector(self, word):
        return word2vec_model[word] if word in word2vec_model else np.zeros(word2vec_model.vector_size)

    def get_vector_bert(self, dic, word):
      # Sua lógica para obter o vetor BERT
      return dic[word] if word in dic else np.zeros(len(next(iter(dic.values()))))

    def getCode(self):
        # Implementação do método getCode
        code = f"ma{self.autoInc}"
        self.autoInc += 1
        return code

    def addConcept(self, name: str, code: str, pos: str):
        # Verifica se o conceito já existe e adiciona uma nova instância se necessário
        for i in range(self.conceptCount):
            if self.nodes[i]['code'] == code:
                nCode = self.getCode()
                self.addConcept(name, nCode, pos)
                rName = self.getCode()
                self.addRelationIn("equality", rName, code)
                self.addRelationEx("equality", rName, nCode)
                return

        # Expandir a lista se necessário
        if self.conceptCount >= len(self.nodes):
            self.nodes.append(None)

        # Adicionar o novo conceito
        self.nodes[self.conceptCount] = {'lemma': name, 'code': code, 'pos': pos}
        self.conceptCount += 1

    def addRelationIn(self, name: str, code: str, in_relation: str):
        found = False
        for i in range(self.edgeCount):
            if self.edges[i]['code'] == code:
                found = True
                self.edges[i]['in'] = in_relation
                break
        if not found:
            if self.edgeCount >= len(self.edges):
                self.edges.append(None)  # Expande a lista
            self.edges[self.edgeCount] = {'code': code, 'name': name, 'in': in_relation}
            self.edgeCount += 1

    def addRelationEx(self, name: str, code: str, ex: str):
        found = False
        for i in range(self.edgeCount):
            if self.edges[i]['code'] == code:
                found = True
                self.edges[i]['ex'] = ex
                break
        if not found:
            if self.edgeCount >= len(self.edges):
                self.edges.append(None)  # Expande a lista
            self.edges[self.edgeCount] = {'code': code, 'name': name, 'ex': ex}
            self.edgeCount += 1

    def addIsRelation(self, name: str, code: str):
        found = False
        for i in range(self.edgeCount):
            if self.edges[i]['code'] == code:
                self.edges[i]['name'] = name
                return
        self.edges[self.edgeCount] = {'code': code, 'name': name}
        self.edgeCount += 1

    def getWeight(self, relation: str) -> int:
        if relation == "equality":
            return 1
        elif relation == "is":
            return 4
        elif relation == "rep":
            return 1
        elif relation == "for":
            return 1
        elif relation == "on":
            return 1
        elif relation == "of":
            return 1
        elif relation == "in":
            return 3
        elif relation == "experiencer":
            return 6
        elif relation == "agent":
            return 8
        elif relation == "theme":
            return 8
        else:
            return 1

    def printNodes(self):
        for i in range(self.conceptCount):
            node = self.nodes[i]
            print(f"{node['code']}   {node['lemma']}   {node['pos']}")

    def printEdges(self):
        for i in range(self.edgeCount):
            edge = self.edges[i]
            print(f"{edge['name']}\t{edge.get('in', '')}\t{edge.get('ex', '')}")

    def getNode(self, code: str):
        for i in range(self.conceptCount):
            if self.nodes[i] and self.nodes[i]['code'] == code:
                return self.nodes[i]

        for i in range(self.edgeCount):
            edge = self.edges[i]
            if edge:
                if 'name' in edge and edge['name'] == "equality":
                    if 'ex' in edge and edge['ex'] == code:
                        in_code = edge.get('in')  # Use .get() to avoid KeyError
                        if in_code:
                            return self.getNode(in_code)
        return None  # Return None or a default value if no node is found

    def finalize(self):
        for i in range(self.edgeCount):
            if self.edges[i]['name'] == "is":
                for j in range(self.edgeCount):
                    if self.edges[i]['code'] == self.edges[j]['code']:
                        self.edges[i]['ex'] = self.edges[j]['ex']
                        self.edges[i]['in'] = self.edges[j]['in']
                        print("found")
                        break

    def checkGraph(self) -> bool:
            if self.debug:
                for i in range(self.conceptCount):
                    node = self.nodes[i]
                    if node:
                        print(f"{node['code']}   {node['lemma']}")

            for i in range(self.edgeCount):
                print(i)
                edge = self.edges[i]
                print(edge)
                if edge:
                    in_node_obj = self.getNode(edge.get('in'))
                    ex_node_obj = self.getNode(edge.get('ex'))

                    in_node = in_node_obj['lemma'] if in_node_obj is not None else None
                    ex_node = ex_node_obj['lemma'] if ex_node_obj is not None else None

                    print(f"{in_node}  {edge['name']}   {ex_node}")

                    if edge['name'] == "equality" and in_node == ex_node:
                        self.repeated += 1

                    for j in range(self.edgeCount):
                        other_edge = self.edges[j]
                        if other_edge and edge['name'] == other_edge['name']:
                            in_node_other_obj = self.getNode(other_edge.get('in'))
                            ex_node_other_obj = self.getNode(other_edge.get('ex'))
                            in_node_other = in_node_other_obj['lemma'] if in_node_other_obj is not None else None
                            ex_node_other = ex_node_other_obj['lemma'] if ex_node_other_obj is not None else None

                            if in_node == in_node_other and ex_node == ex_node_other:
                                other_edge['name'] = "rep"
                                self.repeated += 1

            return True if self.repeated > 0 else False

    def matchRelationNameSTS(self, r1, r2):
        print(f"{r1}    {r2}")
        if r1 == "rep" or r2 == "rep":
            return 0.5
        if r1 == "topic" or r2 == "topic":
            return 0.3
        if r1 == "is" and r2 == "is":
            return 0.7
        if r1 == "equality" and r2 == "equality":
            return 0.7
        if r1 == "rep":
            return 0.0
        if r1 == r2:
            return 1
        if (r1 == "experiencer" and r2 == "of") or (r1 == "of" and r2 == "experiencer"):
            return 0.8
        if (r1 == "theme" and r2 == "agent") or (r1 == "agent" and r2 == "theme"):
            return 0.6
        else:
            return 0.79 - 0.01 * (self.getWeight(r1) + self.getWeight(r2))

    # Convertendo a função matchRelationName para Python
    def matchRelationName(self, r1: str, r2: str) -> float:
        if r1 == "is" and r2 == "is":
            return 0.7
        if r1 == "equality" and r2 == "equality":
            return 0.7
        if r1 == "rep":
            return 0.0
        if r1 == r2:
            return 1.0
        else:
            return 0.73


    # Corrigindo a função stringSim uma última vez
    def stringSim(self, s1: str, s2: str) -> float:
        m, n = len(s1), len(s2)

        if m == 0 and n == 0: return 1.0
        if m == 0 or n == 0: return 0.0

        costs = [0] * (n + 1)

        # Initialize the array
        for k in range(n + 1):
            costs[k] = k

        for i, c1 in enumerate(s1):
            costs[0] = i + 1
            corner = i

            for j, c2 in enumerate(s2):
                upper = costs[j + 1]
                if c1 == c2:
                    costs[j + 1] = corner
                else:
                    t = min(upper, corner)
                    costs[j + 1] = min(costs[j], t) + 1
                corner = upper

        result = costs[n]
        return 1 - float(result) / max(m, n)

    def expand(self, word):
      ewords = [word]
      for edge in self.edges:
          if edge and edge['name'] in ["equality", "is", "of"]:
              node_ex = self.getNode(edge['ex'])
              node_in = self.getNode(edge['in'])

              if node_ex and node_ex['lemma'] == word:
                  ewords.append(node_in['lemma']) if node_in else None
              elif node_in and node_in['lemma'] == word:
                  ewords.append(node_ex['lemma']) if node_ex else None

      return ewords


#GraphSTSBert

In [ ]:
class GraphSTSBert(Graph):
    def __init__(self, sentence: str, debug: bool = False):
        super().__init__(sentence, debug)

    def matchConceptBert(self, dic1, dic2, c1, c2):
        vec1 = self.get_vector_bert(dic1, c1)
        vec2 = self.get_vector_bert(dic2, c2)
        # Similaridade do cosseno
        return 1 - cosine(vec1, vec2) if np.any(vec1) and np.any(vec2) else 0

    def matchConceptWithExpansionBert(self, dic1, dic2, g1, c1, g2, c2):
        all1 = g1.expand(c1)
        all2 = g2.expand(c2)
        print("expansions:", all1, all2)
        max_similarity = 0

        for word1 in all1:
            for word2 in all2:
                similarity = self.matchConceptBert(dic1, dic2, word1, word2)
                if similarity > max_similarity:
                    max_similarity = similarity

        return max_similarity

    def matchRelationBert(self, dic1, dic2, g1, r1, g2, r2):
        node_g1_in = g1.getNode(r1['in'])
        node_g1_ex = g1.getNode(r1['ex'])
        node_g2_in = g2.getNode(r2['in'])
        node_g2_ex = g2.getNode(r2['ex'])

        # Verifica se algum dos nós é None e retorna 0 em tal caso
        if not all([node_g1_in, node_g1_ex, node_g2_in, node_g2_ex]):
            return 0

        node_in_g1 = node_g1_in['lemma']
        node_ex_g1 = node_g1_ex['lemma']
        node_in_g2 = node_g2_in['lemma']
        node_ex_g2 = node_g2_ex['lemma']

        # d1 = self.matchConceptWithExpansionBert(dic1, dic2, g1, node_in_g1, g2, node_in_g2)
        # d2 = self.matchConceptWithExpansionBert(dic2, dic2, g1, node_ex_g1, g2, node_ex_g2)
        d1 = self.matchConceptBert(dic1, dic2, node_in_g1, node_in_g2)
        d2 = self.matchConceptBert(dic1, dic2, node_ex_g1, node_ex_g2)
        d3 = self.matchRelationNameSTS(r1['name'], r2['name'])

        return ((d2 + d1) / 2.0) * d3

    def matchGraphBert(self, dic1, dic2, g):
        matrix = np.zeros((self.edgeCount, g.edgeCount))

        for i in range(self.edgeCount):
            for j in range(g.edgeCount):
                similarity = self.matchRelationBert(dic1, dic2, self, self.edges[i], g, g.edges[j])
                matrix[i, j] = similarity

        sum_d1, sum_d2, weight_sum_d1, weight_sum_d2 = 0, 0, 0, 0
        nan_count_d1, nan_count_d2 = 0, 0

        for i in range(self.edgeCount):
            max_val = np.max(matrix[i, :])
            if max_val == 0:
                nan_count_d1 += 1
            else:
                rel = g.edges[np.argmax(matrix[i, :])]['name']
                weight = self.getWeight(rel)
                sum_d1 += max_val * weight
                weight_sum_d1 += weight

        for j in range(g.edgeCount):
            max_val = np.max(matrix[:, j])
            if max_val == 0:
                nan_count_d2 += 1
            else:
                rel = self.edges[np.argmax(matrix[:, j])]['name']
                weight = self.getWeight(rel)
                sum_d2 += max_val * weight
                weight_sum_d2 += weight

        d1 = sum_d1 / weight_sum_d1 if weight_sum_d1 > 0 else 0
        d2 = sum_d2 / weight_sum_d2 if weight_sum_d2 > 0 else 0

        g1.printRelationMatrix(matrix, g1, g2, export_csv=True, filename="relation_matrix.csv")

        return (d1 + d2) / 2 if self.edgeCount - nan_count_d1 > 0 and g.edgeCount - nan_count_d2 > 0 else 0


    def getNodeLemma(self, code):
        node = self.getNode(code)
        return node['lemma'] if node else 'N/A'

    def printRelationMatrix(self, matrix, g1, g2, export_csv=False, filename="relation_matrix.csv"):
        print("Matriz de Relações:")
        header = [""] + [f"{g2.edges[j]['name']} ({self.getNodeLemma(g2.edges[j]['in'])} - {self.getNodeLemma(g2.edges[j]['ex'])})" for j in range(g2.edgeCount)]
        print("     ".join(header))

        rows = []
        for i in range(self.edgeCount):
            row = [f"{g1.edges[i]['name']} ({self.getNodeLemma(g1.edges[i]['in'])} - {self.getNodeLemma(g1.edges[i]['ex'])})"]
            for j in range(g2.edgeCount):
                cell_value = f"{matrix[i, j]:.4f} ({self.getNodeLemma(g1.edges[i]['in'])} - {self.getNodeLemma(g1.edges[i]['ex'])}, {self.getNodeLemma(g2.edges[j]['in'])} - {self.getNodeLemma(g2.edges[j]['ex'])})"
                row.append(cell_value)
            rows.append(row)

        # Imprimir a matriz de relações
        for row in rows:
            print("     ".join(row))

        # Exportar para CSV se necessário
        if export_csv:
            with open(filename, 'w', newline='') as csvfile:
                csvwriter = csv.writer(csvfile)
                csvwriter.writerow(header)
                csvwriter.writerows(rows)
            print(f"Matriz exportada para {filename}")


    def calculateGraph(self):
        matrix = np.zeros((self.edgeCount, g.edgeCount))
        for j in range(self.edgeCount):
            max_val = np.max(matrix[:, j])
            if max_val == 0:
                nan_count_d2 += 1
            else:
                rel = self.edges[np.argmax(matrix[:, j])]['name']
                weight = self.getWeight(rel)
                sum_d2 += max_val * weight
                weight_sum_d2 += weight

#Define função similaridade dos grafos

In [ ]:
class GraphSTS(Graph):
    def __init__(self, sentence: str, debug: bool = False):
        super().__init__(sentence, debug)

    def matchConcept(self, c1, c2):
        vec1 = self.get_vector(c1)
        vec2 = self.get_vector(c2)
        # Similaridade do cosseno
        return 1 - cosine(vec1, vec2) if np.any(vec1) and np.any(vec2) else 0

    def matchConceptWithExpansion(self, g1, c1, g2, c2):
        all1 = g1.expand(c1)
        all2 = g2.expand(c2)
        print("expansions:", all1, all2)
        max_similarity = 0

        for word1 in all1:
            for word2 in all2:
                similarity = self.matchConcept(word1, word2)
                if similarity > max_similarity:
                    max_similarity = similarity

        return max_similarity

    def matchRelation(self, g1, r1, g2, r2):
        node_g1_in = g1.getNode(r1['in'])
        node_g1_ex = g1.getNode(r1['ex'])
        node_g2_in = g2.getNode(r2['in'])
        node_g2_ex = g2.getNode(r2['ex'])

        # Verifica se algum dos nós é None e retorna 0 em tal caso
        if not all([node_g1_in, node_g1_ex, node_g2_in, node_g2_ex]):
            return 0

        node_in_g1 = node_g1_in['lemma']
        node_ex_g1 = node_g1_ex['lemma']
        node_in_g2 = node_g2_in['lemma']
        node_ex_g2 = node_g2_ex['lemma']

        # d1 = self.matchConceptWithExpansion(g1, node_in_g1, g2, node_in_g2)
        # d2 = self.matchConceptWithExpansion(g1, node_ex_g1, g2, node_ex_g2)
        d1 = self.matchConcept(node_in_g1, node_in_g2)
        d2 = self.matchConcept(node_ex_g1, node_ex_g2)
        d3 = self.matchRelationNameSTS(r1['name'], r2['name'])

        return ((d2 + d1) / 2.0) * d3

    def calculateGraph(self):
        matrix = np.zeros((self.edgeCount, g.edgeCount))
        for j in range(self.edgeCount):
            max_val = np.max(matrix[:, j])
            if max_val == 0:
                nan_count_d2 += 1
            else:
                rel = self.edges[np.argmax(matrix[:, j])]['name']
                weight = self.getWeight(rel)
                sum_d2 += max_val * weight
                weight_sum_d2 += weight

    def printRelationMatrix(self, matrix, g, export_csv=False, filename="relation_matrix.csv"):
        print("Matriz de Relações:")
        print("     ", end="")
        header = [""] + [g.edges[j]['name'] for j in range(g.edgeCount)]
        print("     ".join(header))

        rows = []
        for i in range(self.edgeCount):
            row = [self.edges[i]['name']]
            print(f"{self.edges[i]['name']:>12}", end="     ")
            for j in range(g.edgeCount):
                print(f"{matrix[i, j]:>12.4f}", end="     ")
                row.append(matrix[i, j])
            rows.append(row)
            print()

        if export_csv:
            with open(filename, 'w', newline='') as csvfile:
                csvwriter = csv.writer(csvfile)
                csvwriter.writerow(header)
                csvwriter.writerows(rows)
            print(f"Matriz exportada para {filename}")

    def matchGraph(self, g):
        matrix = np.zeros((self.edgeCount, g.edgeCount))

        for i in range(self.edgeCount):
            for j in range(g.edgeCount):
                similarity = self.matchRelation(self, self.edges[i], g, g.edges[j])
                matrix[i, j] = similarity

        sum_d1, sum_d2, weight_sum_d1, weight_sum_d2 = 0, 0, 0, 0
        nan_count_d1, nan_count_d2 = 0, 0

        for i in range(self.edgeCount):
            max_val = np.max(matrix[i, :])
            if max_val == 0:
                nan_count_d1 += 1
            else:
                rel = g.edges[np.argmax(matrix[i, :])]['name']
                weight = self.getWeight(rel)
                sum_d1 += max_val * weight
                weight_sum_d1 += weight

        for j in range(g.edgeCount):
            max_val = np.max(matrix[:, j])
            if max_val == 0:
                nan_count_d2 += 1
            else:
                rel = self.edges[np.argmax(matrix[:, j])]['name']
                weight = self.getWeight(rel)
                sum_d2 += max_val * weight
                weight_sum_d2 += weight

        d1 = sum_d1 / weight_sum_d1 if weight_sum_d1 > 0 else 0
        d2 = sum_d2 / weight_sum_d2 if weight_sum_d2 > 0 else 0

        self.printRelationMatrix(matrix, g)

        return (d1 + d2) / 2 if self.edgeCount - nan_count_d1 > 0 and g.edgeCount - nan_count_d2 > 0 else 0

In [ ]:
# # Exemplo de uso
# graph = GraphSTS("A cock is an adult male chicken")
# idf_value = graph.get_idf("is")
# print(f"Valor IDF para 'cock': {idf_value}")

#Gera os grafos a partir da classe

In [ ]:
def generate_graphs(fres):
    # Regular expressions
    conceptReg = re.compile(r"c\d+:(.+?):.+instance (.+?) ")
    argReg = re.compile(r"c\d+:(.+?):.+arg (.+?) ")
    inRelReg = re.compile(r"(.*?):(.*?):.* int (.*?) ")
    exRelReg = re.compile(r"(.*?):(.*?):.* ext (.*?) ")
    inEqRelReg = re.compile(r"(.*?):equality int (.*?) ")
    exEqRelReg = re.compile(r"(.*?):equality ext (.*?) ")
    isReg = re.compile(r".* (.*?):equality \d+ . is ")
    refReg = re.compile(r".+referent (.+?) .* (.+?) ")

    for i in range(3):
      sentence = fres.readline()

    sentence = sentence.replace("%%% ", "")
    g1 = GraphSTSBert(sentence, True)
    # g1 = GraphSTS(sentence, True)

    # print(g1.words)
    for line in fres:
        # Initialize your GraphSTS object here (Placeholder)
        if line and line[0] == '%':
            continue
        else:
            match = conceptReg.search(line)
        if match:
            g1.addConcept(match.group(1), match.group(2), "")
            pass
        elif (match := argReg.search(line)):
            g1.addConcept(match.group(1), match.group(2), "jj")
            pass
        elif (match := inRelReg.search(line)):
            g1.addRelationIn(match.group(2), match.group(1), match.group(3))
            pass
        elif (match := exRelReg.search(line)):
            g1.addRelationEx(match.group(2), match.group(1), match.group(3))
            pass
        elif (match := inEqRelReg.search(line)):
            g1.addRelationIn("equality", match.group(1), match.group(2))
            pass
        elif (match := exEqRelReg.search(line)):
            g1.addRelationEx("equality", match.group(1), match.group(2))
            pass
        elif (match := isReg.search(line)):
            g1.addIsRelation("is", match.group(1))
            pass
        elif (match := refReg.search(line)):
            temp = match.group(2)
            if len(temp) > 3:
                g1.addConcept(match.group(2), match.group(1), "ref")
                pass

    return g1

# Processa usando BERT

In [ ]:
# Função para obter embeddings de uma sentença
def get_sentence_embeddings(sentence):
  # Tokeniza a sentença
  inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=512)

  # Obtém os embeddings
  with torch.no_grad():
      outputs = model(**inputs)

  # Mapeia tokens para palavras
  token_embeddings = outputs.last_hidden_state.squeeze().numpy()
  tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

  # Cria um dicionário para mapear palavras aos seus embeddings
  word_embeddings = {}
  current_word = ""
  word_embedding = None

  for i, token in enumerate(tokens):
      # Se é o primeiro subtoken de uma nova palavra
      if token.startswith("##"):
          token = token[2:]  # Remove o prefixo '##'
          current_word += token
          word_embedding += token_embeddings[i]
      else:
          if current_word:
              # Adiciona a palavra anterior ao dicionário
              word_embeddings[current_word] = word_embedding / len(current_word.split("##"))
          current_word = token
          word_embedding = token_embeddings[i]

  # Adiciona a última palavra
  if current_word:
      word_embeddings[current_word] = word_embedding / len(current_word.split("##"))

  return word_embeddings

#Importa dataset

In [ ]:
# Nome do arquivo
NOME_ARQUIVO_ORIGINAL_COMPACTADO = "dataset.csv"

In [ ]:
# Se estiver executando no Google Colaboratory
if IN_COLAB:

  !cp "$DIRETORIO_DRIVE$NOME_ARQUIVO_ORIGINAL_COMPACTADO" "$DIRETORIO_LOCAL"

  logging.info("Terminei a cópia.")

#Processa dataset

In [ ]:
import pandas as pd
import os
import subprocess

# Função para processar sentenças e gerar os grafos
def process_sentence(sentence, file_name):
    # Grava a sentença em um arquivo temporário
    with open(file_name, 'w') as file:
        file.write(sentence)

    # Chama o CandC e o Boxer
    subprocess.run(f'echo "{sentence}" | bin/candc --models models/boxer --candc-printer boxer --output /content/CANDC/{file_name}_candc', shell=True)
    subprocess.run(f'bin/boxer --input /content/CANDC/{file_name}_candc --output /content/CANDC/{file_name}_drg --semantics drg', shell=True)



In [ ]:
# Lê o arquivo CSV
df = pd.read_csv('/content/CANDC/dataset.csv')
# df = df[58:59]
df

,Sentence_Pair,Sentence_1,Sentence_2
0,1.cord:smile,"Cord is strong, thick string.",A smile is the expression that you have on you...
1,2.rooster:voyage,A rooster is an adult male chicken.,A voyage is a long journey on a ship or in a s...
2,3.noon:string,Noon is 12 o'clock in the middle of the day.,"String is thin rope made of twisted threads, u..."
3,4.fruit:furnace,Fruit or a fruit is something which grows on a...,A furnace is a container or enclosed space in ...
4,5.autograph:shore,An autograph is the signature of someone famou...,"The shores or shore of a sea, lake or wide riv..."
...,...,...,...
60,61.cushion:pillow,A cushion is a fabric case filled with soft ma...,A pillow is a rectangular cushion which you re...
61,62.cemetery:graveyard,A cemetery is a place where dead people's bodi...,"A graveyard is an area of land, sometimes near..."
62,63.automobile:car,An automobile is a car.,A car is a motor vehicle with room for a small...
63,64.midday:noon,Midday is 12 o'clock in the middle of the day.,Noon is 12 o'clock in the middle of the day.


##Processa usando word2vec

In [ ]:
import re

# Função para limpar a sentença removendo pontos e vírgulas
def clean_sentence(sentence):
    return re.sub(r'[.,]', '', sentence)

# Lista para armazenar os resultados de similaridade
similarities = []

for index, row in df.iterrows():
    # Limpa e processa as duas sentenças
    sentence1_clean = clean_sentence(row['Sentence_1'])
    process_sentence(sentence1_clean, 'sentence1')
    print(sentence1_clean)
    dict1 = get_sentence_embeddings(sentence1_clean)
    sentence2_clean = clean_sentence(row['Sentence_2'])
    process_sentence(sentence2_clean, 'sentence2')
    print(sentence2_clean)
    dict2 = get_sentence_embeddings(sentence2_clean)

    # Abre os arquivos processados
    with open("/content/CANDC/sentence1_drg", "r") as fres, open("/content/CANDC/sentence2_drg", "r") as fres2:
        # Gera os grafos
        g1 = generate_graphs(fres)
        g2 = generate_graphs(fres2)

        # Calcula e armazena a similaridade
        similarity = g1.matchGraphBert(dict1, dict2, g2)
        # similarity = g1.matchGraph(g2)
        similarities.append(similarity)

# Adiciona a coluna de similaridade ao DataFrame
df['Similarity'] = similarities

# Salva o DataFrame modificado como CSV
# df.to_csv('/content/CANDC/results.csv', index=False)


Cord is strong thick string
A smile is the expression that you have on your face when you are pleased or amused or when you are being friendly
Cord is strong thick string 

A smile is the expression that you have on your face when you are pleased or amused or when you are being friendly 

Theme    Theme
Theme    Theme
Theme    equality
Theme    Actor
Theme    of
Theme    Theme
Theme    is
Theme    equality
Theme    Actor
Theme    of
Theme    Theme
Theme    Theme
Theme    on
Theme    on
Theme    Theme
Theme    Theme
Theme    equality
Theme    Actor
Theme    of
Theme    Theme
Theme    is
Theme    equality
Theme    Actor
Theme    of
Theme    Theme
Theme    Theme
Theme    on
Theme    on
is    Theme
is    Theme
is    equality
is    Actor
is    of
is    Theme
is    is
is    equality
is    Actor
is    of
is    Theme
is    Theme
is    on
is    on
Matriz de Relações:
     Theme (N/A - N/A)     Theme (N/A - N/A)     equality (N/A - N/A)     Actor (N/A - N/A)     of (N/A - N/A)     Theme (N/A - N

In [ ]:
similarities

[0.2090099872010095,
 0.2596228754073381,
 0.15831176429454769,
 0.31920888561113137,
 0.24004398056678475,
 0.2729843339920044,
 0.3027580660400646,
 0.3878445852920413,
 0.26134298867526207,
 0.3533485798131336,
 0.273022679798305,
 0.24155203537705044,
 0.3062128094881773,
 0.3912428460425728,
 0.3727705158936706,
 0.29145501226186754,
 0.43156221519224347,
 0.3662818387026588,
 0.2650926766482492,
 0.3065923188990829,
 0.38641736042499547,
 0.2433099226653576,
 0.3596692054303629,
 0.37250331249708935,
 0.3775070300524111,
 0.12237958434969187,
 0.20391207878788312,
 0.3413080347568861,
 0.20720813139208724,
 0.5132121476419269,
 0.39104915309540866,
 0.24993409044685816,
 0.2719219016085068,
 0.23849486002077658,
 0.253671829521656,
 0.394311330200961,
 0.31522882334662206,
 0.3949367370722549,
 0.3991733197371165,
 0.32512656950950625,
 0.29200497647036205,
 0.28375962837326985,
 0.3190937677238669,
 0.26410179926082494,
 0.2803592672881981,
 0.3029579643625766,
 0.43226410708730

In [ ]:
df


,Sentence_Pair,Sentence_1,Sentence_2,Similarity
0,1.cord:smile,"Cord is strong, thick string.",A smile is the expression that you have on you...,0.209010
1,2.rooster:voyage,A rooster is an adult male chicken.,A voyage is a long journey on a ship or in a s...,0.259623
2,3.noon:string,Noon is 12 o'clock in the middle of the day.,"String is thin rope made of twisted threads, u...",0.158312
3,4.fruit:furnace,Fruit or a fruit is something which grows on a...,A furnace is a container or enclosed space in ...,0.319209
4,5.autograph:shore,An autograph is the signature of someone famou...,"The shores or shore of a sea, lake or wide riv...",0.240044
...,...,...,...,...
60,61.cushion:pillow,A cushion is a fabric case filled with soft ma...,A pillow is a rectangular cushion which you re...,0.400597
61,62.cemetery:graveyard,A cemetery is a place where dead people's bodi...,"A graveyard is an area of land, sometimes near...",0.490750
62,63.automobile:car,An automobile is a car.,A car is a motor vehicle with room for a small...,0.421686
63,64.midday:noon,Midday is 12 o'clock in the middle of the day.,Noon is 12 o'clock in the middle of the day.,0.647569


In [ ]:
df.to_csv('/content/CANDC/results.csv', index=False)